## Mount Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')
# Set the path to the folder where your model or dataset is located
model_path = '/content/drive/MyDrive/XpertScan/MobileNet.keras'
test_data_path = '/content/drive/MyDrive/XpertScan/test'

Mounted at /content/drive


## Import Libraries

In [3]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import load_model
import seaborn as sns
import matplotlib.pyplot as plt


## Load Modal

In [4]:
# Load the model
loaded_model = load_model(model_path)

# Check the model summary to ensure it loaded correctly
loaded_model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,658,580 (21.59 MB)

 Trainable params: 1,535,558 (5.86 MB)

 Non-trainable params: 1,051,904 (4.01 MB)

 Optimizer params: 3,071,118 (11.72 MB)

## Prepare Data

In [5]:
# Create an ImageDataGenerator for test data with rescaling
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Create the test data generator
test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(224, 224),  # Resize images to 224x224 for MobileNet
    batch_size=32,           # Set the batch size
    class_mode='sparse',     # Use 'sparse' for integer-encoded labels
    shuffle=False            # Don't shuffle for evaluation (keeps the order)
)

# Check the class labels in the dataset
print(test_generator.class_indices)


Found 558 images belonging to 6 classes.
{'COPD': 0, 'Covid19': 1, 'Lung_Metastasis': 2, 'Normal': 3, 'Pneumonia': 4, 'Tuberculosis': 5}


## Evaluate Modal

In [6]:
# Using sparse categorical crossentropy if labels are integer encoded
loaded_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Now evaluate the model
loss, accuracy = loaded_model.evaluate(test_generator)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy * 100:.2f}%")



/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


18/18 ━━━━━━━━━━━━━━━━━━━━ 316s 18s/step - accuracy: 0.2455 - loss: 1.8577
Test Loss: 1.9004
Test Accuracy: 19.71%


## Generate Confusion Matrix

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


true_classes = []
predicted_classes = []

# Loop through the test dataset to get true labels and predicted labels
for i, (x_batch, y_batch) in enumerate(test_generator):
    # Make predictions on the batch
    predictions = loaded_model.predict(x_batch)
    predicted_classes_batch = np.argmax(predictions, axis=1)

    # If y_batch is one-hot encoded, convert to integer labels
    if y_batch.ndim > 1:  # Check if y_batch is one-hot encoded (has more than 1 dimension)
        true_classes.extend(np.argmax(y_batch, axis=1))  # Convert one-hot to integer labels
    else:
        true_classes.extend(y_batch)  # If already integer encoded, use directly

    predicted_classes.extend(predicted_classes_batch)

# Now true_classes and predicted_classes are populated with the actual and predicted labels
# Generate confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Define class labels corresponding to your integer encoding
class_labels = ['COPD', 'Covid19', 'Lung_Metastasis', 'Normal', 'Pneumonia', 'Tuberculosis']
# Replace with your actual class names (ensure there is no trailing space in 'Pneumonia')

# Plot confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix of MobileNet')
plt.show()


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 

In [ ]:
from sklearn.metrics import (
    classification_report, precision_score, recall_score,
    f1_score, roc_auc_score, roc_curve
)
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import numpy as np

# Convert to numpy arrays
true_classes = np.array(true_classes)
predicted_classes = np.array(predicted_classes)

# Optional: Get predicted probabilities if needed for ROC
predicted_probs = []
for x_batch, _ in test_generator:
    preds = loaded_model.predict(x_batch)
    predicted_probs.extend(preds)
    if len(predicted_probs) >= test_generator.samples:
        break

predicted_probs = np.array(predicted_probs)

# Define your class labels
class_labels = ['COPD', 'Covid19', 'Lung_Metastasis', 'Normal', 'Pneumonia', 'Tuberculosis']

# ----------------------------
# 📊 Classification Report
# ----------------------------
print("Classification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

# ----------------------------
# 🧮 Precision, Recall, F1 (Macro Average)
# ----------------------------
precision = precision_score(true_classes, predicted_classes, average='macro')
recall = recall_score(true_classes, predicted_classes, average='macro')
f1 = f1_score(true_classes, predicted_classes, average='macro')

print(f"\n Precision (macro): {precision:.4f}")
print(f"Recall (macro):    {recall:.4f}")
print(f"F1 Score (macro):  {f1:.4f}")

# ----------------------------
# 📈 ROC-AUC Score (One-vs-Rest for Multiclass)
# ----------------------------
y_true_bin = label_binarize(true_classes, classes=range(len(class_labels)))  # e.g. [0,1,2,3,4,5]
roc_auc = roc_auc_score(y_true_bin, predicted_probs, average='macro', multi_class='ovr')
print(f"ROC-AUC Score (macro OvR): {roc_auc:.4f}")

## ROC Graph

In [ ]:
# Plot ROC curve for each class
fpr = {}
tpr = {}
plt.figure(figsize=(10, 8))

for i in range(len(class_labels)):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], predicted_probs[:, i])
    plt.plot(fpr[i], tpr[i], label=f'{class_labels[i]} (AUC = {roc_auc_score(y_true_bin[:, i], predicted_probs[:, i]):.2f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multiclass ROC Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()